In [2]:
import os
from dotenv import load_dotenv
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.google_genai import GoogleGenAI

from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter

load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

documents = SimpleDirectoryReader('../data').load_data()
model = GoogleGenAI(model='gemini-2.5-flash-preview-05-20', api_key=google_api_key)
model_embedding = GoogleGenAIEmbedding(model_name='text-embedding-004')

Settings.llm = model
Settings.embed_model = model_embedding
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900


index = VectorStoreIndex.from_documents(
        documents, 
        embed_model=model_embedding,
        transformations=[Settings.node_parser]
)



In [3]:
index.storage_context.persist(persist_dir="./gemini_index")

In [11]:
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.core import load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="./gemini_index")
index = load_index_from_storage(storage_context)


query_engine = index.as_query_engine()
def ask_gemini(query):
    response = query_engine.query(query)
    return response.response
print(ask_gemini("What is the purpose of the document?"))

The document addresses the critical issue of water scarcity and the importance of efficient irrigation in global agriculture. It highlights agriculture's significant water footprint and the projected increase in demand for irrigated food production, emphasizing the need for effective water resource management in the face of climate change. The content details various machine learning models, such as XGBoost, CNN, ANN, and DNN, used for prediction, along with discussions on dataset summaries, model evaluation, and user interface aspects.
